In [2]:
import os ; os.chdir(r"D:\Git_repos\Photonic_GCN")
from constants import edges_table, nodes_groups
# !pip install torch torch-geometric
import torch , pandas as pd
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.utils.data import TensorDataset, random_split , DataLoader as DataLoader_primary
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [3]:
def edges_table_to_list(edges_table):
    edges = []
    for key in edges_table.keys():
        nodes_list = edges_table[key]
        for node in nodes_list:
            if ((key, node) not in edges) and ((node, key) not in edges):
                edges.append((key, node))
    return edges

In [4]:
edge_list = edges_table_to_list(edges_table)  # 107 edges
print(len(edge_list)) 
print(edge_list)

107
[('N69', 'N70'), ('N70', 'N71'), ('N71', 'N72'), ('N72', 'N73'), ('N73', 'N74'), ('N73', 'N60'), ('N74', 'N60'), ('N74', 'N94'), ('N60', 'N45'), ('N60', 'N46'), ('N60', 'N47'), ('N60', 'N61'), ('N60', 'N75'), ('N41', 'N42'), ('N42', 'N43'), ('N43', 'N44'), ('N44', 'N45'), ('N45', 'N46'), ('N46', 'N33'), ('N87', 'N86'), ('N86', 'N85'), ('N85', 'N84'), ('N84', 'N83'), ('N83', 'N82'), ('N83', 'N68'), ('N82', 'N68'), ('N82', 'N94'), ('N68', 'N55'), ('N68', 'N54'), ('N68', 'N53'), ('N68', 'N67'), ('N68', 'N81'), ('N59', 'N58'), ('N58', 'N57'), ('N57', 'N56'), ('N56', 'N55'), ('N55', 'N54'), ('N54', 'N34'), ('N33', 'N61'), ('N33', 'N47'), ('N33', 'N48'), ('N34', 'N52'), ('N34', 'N53'), ('N34', 'N67'), ('N94', 'N61'), ('N94', 'N75'), ('N94', 'N76'), ('N95', 'N80'), ('N95', 'N81'), ('N95', 'N82'), ('N95', 'N67'), ('N67', 'N81'), ('N67', 'N53'), ('N67', 'N80'), ('N67', 'N66'), ('N67', 'N52'), ('N53', 'N52'), ('N53', 'N66'), ('N81', 'N80'), ('N81', 'N66'), ('N80', 'N79'), ('N80', 'N66'), ('N

In [5]:
node_names = list(edges_table.keys())  # 51 nodes
print(node_names)

['N69', 'N70', 'N71', 'N72', 'N73', 'N74', 'N60', 'N41', 'N42', 'N43', 'N44', 'N45', 'N46', 'N87', 'N86', 'N85', 'N84', 'N83', 'N82', 'N68', 'N59', 'N58', 'N57', 'N56', 'N55', 'N54', 'N33', 'N34', 'N94', 'N95', 'N67', 'N53', 'N81', 'N80', 'N52', 'N79', 'N51', 'N65', 'N66', 'N78', 'N64', 'N50', 'N61', 'N75', 'N47', 'N62', 'N76', 'N48', 'N63', 'N77', 'N49']


In [6]:
# Node features (3 features per node)
x = torch.randn((51, 3))  # Random initialization of features for simplicity

# Network label (one-hot encoded for 37 categories)
network_label = torch.randint(0, 37, (1,)).item()
# y = F.one_hot(torch.tensor(network_label), num_classes=37).float()
y = torch.tensor([network_label])
y

tensor([9])

In [7]:
print(x.shape)
x

torch.Size([51, 3])


tensor([[ 0.1627,  0.3015,  0.4168],
        [ 1.2500, -1.8819,  0.0663],
        [-0.3749, -0.1705,  0.1146],
        [-0.4645, -0.3018,  0.9051],
        [ 0.0160, -0.8974,  1.3347],
        [ 0.8481,  0.5993, -0.3945],
        [ 1.3775,  1.8439,  1.1732],
        [ 0.3338,  0.8211,  0.0628],
        [ 1.2542, -1.0817,  0.3799],
        [ 1.9548,  0.4099,  0.1426],
        [-1.0719,  0.7022, -0.3511],
        [-0.8283,  0.9441,  1.2259],
        [ 0.9249, -0.2223, -0.5086],
        [-0.8212, -0.0357, -0.2938],
        [ 1.2699,  1.4800, -0.4217],
        [-1.0134, -1.7448,  1.6486],
        [-0.6855,  0.4944, -0.3422],
        [-0.7083,  1.0831,  0.6803],
        [ 1.4261,  0.2388,  0.1203],
        [ 0.3037,  0.7697, -0.7190],
        [ 0.7801, -0.7359,  0.7959],
        [ 0.7569, -0.8239,  0.2688],
        [-0.6070,  0.8700,  1.8457],
        [ 1.3301, -0.2557,  0.7494],
        [-0.9290,  0.3908, -0.6329],
        [-0.2678,  0.6561, -0.3947],
        [ 0.4266,  0.8623,  0.3399],
 

In [8]:
y

tensor([9])

In [9]:
# Convert edge list to indices
edge_index = torch.tensor([[node_names.index(u), node_names.index(v)] for u, v in edge_list], dtype=torch.long).t().contiguous()
edge_index_orignal = edge_index

# Create data object
data = Data(x=x, edge_index=edge_index, y=y)

In [10]:
data

Data(x=[51, 3], edge_index=[2, 107], y=[1])

In [11]:
print(edge_index.shape)
edge_index

torch.Size([2, 107])


tensor([[ 0,  1,  2,  3,  4,  4,  5,  5,  6,  6,  6,  6,  6,  7,  8,  9, 10, 11,
         12, 13, 14, 15, 16, 17, 17, 18, 18, 19, 19, 19, 19, 19, 20, 21, 22, 23,
         24, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30,
         30, 30, 31, 31, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 35, 36, 36,
         36, 36, 37, 37, 37, 37, 39, 39, 39, 40, 40, 40, 40, 41, 41, 42, 42, 42,
         42, 42, 43, 43, 44, 44, 45, 45, 45, 45, 45, 46, 46, 47, 47, 48, 48],
        [ 1,  2,  3,  4,  5,  6,  6, 28, 11, 12, 44, 42, 43,  8,  9, 10, 11, 12,
         26, 14, 15, 16, 17, 18, 19, 19, 28, 24, 25, 31, 30, 32, 21, 22, 23, 24,
         25, 27, 42, 44, 47, 34, 31, 30, 42, 43, 46, 33, 32, 18, 30, 32, 31, 33,
         38, 34, 34, 38, 33, 38, 35, 38, 37, 36, 37, 38, 39, 40, 37, 38, 41, 40,
         37, 38, 40, 39, 38, 41, 49, 48, 40, 48, 49, 50, 41, 50, 48, 43, 46, 45,
         47, 44, 46, 45, 45, 47, 46, 49, 48, 50, 47, 49, 48, 48, 50, 49, 50]])

In [12]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)  # Output 1 feature per node

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(Net, self).__init__()
        self.gcn = GCN(in_channels, hidden_channels)
        self.fc1 = torch.nn.Linear(51, 128)  # 51 nodes, each with 1 feature from GCN
        self.fc2 = torch.nn.Linear(128, 1)   # Output a single value

    def forward(self, data):
        x = self.gcn(data)  # GCN output, shape [num_nodes, 1]
        x = x.view(1, -1)  # Flatten the node features, shape [1, num_nodes]
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x  # No softmax or other activation for regression

# Model parameters
in_channels = 3
hidden_channels = 16

# Initialize model
model = Net(in_channels, hidden_channels)
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
# Training loop
def train(data, model, optimizer, criterion, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out.squeeze(), data.y.float())
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Training the model
train(data, model, optimizer, criterion)


D:\Projects\venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Loss: 79.60138702392578
Epoch 10, Loss: 74.05936431884766
Epoch 20, Loss: 65.26710510253906
Epoch 30, Loss: 49.522178649902344
Epoch 40, Loss: 25.85194969177246
Epoch 50, Loss: 3.616945505142212
Epoch 60, Loss: 1.342528223991394
Epoch 70, Loss: 0.6550459265708923
Epoch 80, Loss: 0.14355044066905975
Epoch 90, Loss: 0.1034700870513916


In [14]:
data

Data(x=[51, 3], edge_index=[2, 107], y=[1])

Data preparation

In [15]:
BATCH_SIZE = 8

In [16]:
df = pd.read_excel('data.xlsx')

X = df.iloc[:, :18].values
y = df.iloc[:, 18:19].values

scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32).to(device)

train_size = int(0.7 * len(X_tensor))
val_size = int(0.15 * len(X_tensor))
test_size = len(X_tensor) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(TensorDataset(X_tensor, y_tensor), [train_size, val_size, test_size])

train_dataloader = DataLoader_primary(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader_primary(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader_primary(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [17]:
X.shape

(8015, 18)

In [18]:
df

,dx1,dx2,dx3,dx4,dx5,dx6,dy1,dy2,dy3,dy4,...,dy6,dr1,dr2,dr3,dr4,dr5,dr6,Q,t_q_max,lambda_q_max
0,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,918.130324,0.554622,0.000001
1,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.750000e-08,2.500000e-08,1846.114296,0.682024,0.000001
2,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,3.000000e-08,2.500000e-08,660.817503,1.089219,0.000002
3,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.750000e-08,2.500000e-08,1846.114296,0.682024,0.000001
4,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,3.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.750000e-08,2.500000e-08,1846.114296,0.682024,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8010,-4.000000e-08,3.500000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,1204.550299,0.521705,0.000001
8011,-4.000000e-08,4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,1306.557266,0.549646,0.000001
8012,-4.000000e-08,4.500000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,618.706366,0.701339,0.000002
8013,-4.000000e-08,5.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,610.208722,0.826126,0.000002


In [19]:
df["Q"][0]

918.1303236596783

In [20]:
nodes2group = dict()

for g in nodes_groups:
    for node in nodes_groups[g]:
        nodes2group[node] = g


In [21]:
xs_list = []
ys_list = []

for n in range(len(df)):
    x = np.zeros((51, 3))
    y = np.array(df["Q"][n])
    for idx, node in enumerate(node_names):
        group = nodes2group[node][1:]
        node_features = np.array((df["dx" + group][n], df["dy" + group][n], df["dr" + group][n],))
        x[idx] = node_features
    xs_list.append(x)
    ys_list.append(y)

data_x = torch.from_numpy(np.stack(xs_list, axis=0))
data_y = torch.from_numpy(np.stack(ys_list, axis=0))
print(data_x.shape)
print(data_y.shape)
print(edge_index.shape)

torch.Size([8015, 51, 3])
torch.Size([8015])
torch.Size([2, 107])


Edge index included in dataloader

In [21]:
import torch
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.loader import DataLoader

class MyGraphDataset(InMemoryDataset):
    def __init__(self, root, x, edge_index, y, transform=None, pre_transform=None):
        self.x = torch.tensor(x, dtype=torch.float)
        self.edge_index = torch.tensor(edge_index, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.float)  # Use float for regression
        super(MyGraphDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = self.process_data()

    def process_data(self):
        data_list = []
        for i in range(len(self.y)):
            x = self.x[i]
            edge_index = self.edge_index
            y = self.y[i]
            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        data, slices = self.collate(data_list)
        return data, slices

    def len(self):
        return len(self.y)

    def get(self, idx):
        data = Data()
        for key in self.data.keys():  # Call keys() method here
            item, slices = self.data[key], self.slices[key]
            start, end = slices[idx].item(), slices[idx + 1].item()
            data[key] = item[start:end] if slices.dim() > 1 else item[slices[idx]:slices[idx + 1]]
        return data

# Initialize your dataset
dataset = MyGraphDataset(root='.', x=data_x, edge_index=edge_index, y=data_y)

# Create a DataLoader from torch_geometric
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check the shapes
for batch in loader:
    print(f'Batch edge_index shape: {batch.edge_index.shape}')
    print(f'Batch x shape: {batch.x.shape}')
    print(f'Batch y shape: {batch.y.shape}')
    break  # Print the shapes of the first batch


Batch edge_index shape: torch.Size([0, 27443360])
Batch x shape: torch.Size([1632, 3])
Batch y shape: torch.Size([32])


C:\Users\User_1\AppData\Local\Temp\ipykernel_23436\2734419112.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x, dtype=torch.float)
C:\Users\User_1\AppData\Local\Temp\ipykernel_23436\2734419112.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.edge_index = torch.tensor(edge_index, dtype=torch.long)
C:\Users\User_1\AppData\Local\Temp\ipykernel_23436\2734419112.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y, dtype=torch.float)  # Use float for regression


In [28]:
edge_index = torch.tensor(edge_index, dtype=torch.long)
edge_index.shape

C:\Users\User_1\AppData\Local\Temp\ipykernel_23436\4184712488.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(edge_index, dtype=torch.long)


torch.Size([2, 107])

In [26]:
edge_index_orignal

tensor([[ 0,  1,  2,  3,  4,  4,  5,  5,  6,  6,  6,  6,  6,  7,  8,  9, 10, 11,
         12, 13, 14, 15, 16, 17, 17, 18, 18, 19, 19, 19, 19, 19, 20, 21, 22, 23,
         24, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30,
         30, 30, 31, 31, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 35, 36, 36,
         36, 36, 37, 37, 37, 37, 39, 39, 39, 40, 40, 40, 40, 41, 41, 42, 42, 42,
         42, 42, 43, 43, 44, 44, 45, 45, 45, 45, 45, 46, 46, 47, 47, 48, 48],
        [ 1,  2,  3,  4,  5,  6,  6, 28, 11, 12, 44, 42, 43,  8,  9, 10, 11, 12,
         26, 14, 15, 16, 17, 18, 19, 19, 28, 24, 25, 31, 30, 32, 21, 22, 23, 24,
         25, 27, 42, 44, 47, 34, 31, 30, 42, 43, 46, 33, 32, 18, 30, 32, 31, 33,
         38, 34, 34, 38, 33, 38, 35, 38, 37, 36, 37, 38, 39, 40, 37, 38, 41, 40,
         37, 38, 40, 39, 38, 41, 49, 48, 40, 48, 49, 50, 41, 50, 48, 43, 46, 45,
         47, 44, 46, 45, 45, 47, 46, 49, 48, 50, 47, 49, 48, 48, 50, 49, 50]])

In [223]:
edge_indices = [edge_index for _ in range(len(data_y))]  # Replace with actual edge indices for each graph

dataset = MyGraphDataset(root='.', x=data_x, edge_index=edge_index, y=data_y)
loader = DataLoader(dataset, batch_size=32, shuffle=True)


C:\Users\User_1\AppData\Local\Temp\ipykernel_22544\4218097030.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x, dtype=torch.float)
C:\Users\User_1\AppData\Local\Temp\ipykernel_22544\4218097030.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.edge_index = torch.tensor(edge_index, dtype=torch.long)
C:\Users\User_1\AppData\Local\Temp\ipykernel_22544\4218097030.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y, dtype=torch.long)


In [224]:
print(loader.dataset.x.shape)
print(loader.dataset.y.shape)
print(loader.dataset.edge_index.shape)
print(loader.dataset.edge_index)

torch.Size([8015, 51, 3])
torch.Size([8015])
torch.Size([2, 107])
tensor([[ 0,  1,  2,  3,  4,  4,  5,  5,  6,  6,  6,  6,  6,  7,  8,  9, 10, 11,
         12, 13, 14, 15, 16, 17, 17, 18, 18, 19, 19, 19, 19, 19, 20, 21, 22, 23,
         24, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30,
         30, 30, 31, 31, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 35, 36, 36,
         36, 36, 37, 37, 37, 37, 39, 39, 39, 40, 40, 40, 40, 41, 41, 42, 42, 42,
         42, 42, 43, 43, 44, 44, 45, 45, 45, 45, 45, 46, 46, 47, 47, 48, 48],
        [ 1,  2,  3,  4,  5,  6,  6, 28, 11, 12, 44, 42, 43,  8,  9, 10, 11, 12,
         26, 14, 15, 16, 17, 18, 19, 19, 28, 24, 25, 31, 30, 32, 21, 22, 23, 24,
         25, 27, 42, 44, 47, 34, 31, 30, 42, 43, 46, 33, 32, 18, 30, 32, 31, 33,
         38, 34, 34, 38, 33, 38, 35, 38, 37, 36, 37, 38, 39, 40, 37, 38, 41, 40,
         37, 38, 40, 39, 38, 41, 49, 48, 40, 48, 49, 50, 41, 50, 48, 43, 46, 45,
         47, 44, 46, 45, 45, 47, 46, 49, 48, 5

In [225]:
# def train(loader, model, optimizer, criterion, epochs=100):
#     model.train()
#     for epoch in range(epochs):
#         total_loss = 0
#         for batch in loader:
#             optimizer.zero_grad()
#             print(f'Batch edge_index shape: {batch.edge_index.shape}')
#             print(f'Batch x shape: {batch.x.shape}')
#             print(f'Batch y shape: {batch.y.shape}')
            
#             out = model(batch)  # Forward pass
#             loss = criterion(out, batch.y)  # Compute loss
#             loss.backward()  # Backward pass
#             optimizer.step()  # Optimize the model
#             total_loss += loss.item()  # Accumulate the loss

#         if epoch % 10 == 0:
#             avg_loss = total_loss / len(loader)  # Compute average loss for the epoch
#             print(f'Epoch {epoch}, Loss: {avg_loss:.4f}')

# train(loader, model, optimizer, criterion, epochs=100)

In [226]:
for batch in loader:
    print(batch)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'torch_geometric.data.data.Data'>

In [227]:
def train(loader, model, optimizer, criterion, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in loader:
            optimizer.zero_grad()
            print(f'Batch edge_index shape: {batch.edge_index.shape}')
            print(f'Batch x shape: {batch.x.shape}')
            print(f'Batch y shape: {batch.y.shape}')
            
            out = model(batch)  # Forward pass
            loss = criterion(out, batch.y)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize the model
            total_loss += loss.item()  # Accumulate the loss

        if epoch % 10 == 0:
            avg_loss = total_loss / len(loader)  # Compute average loss for the epoch
            print(f'Epoch {epoch}, Loss: {avg_loss:.4f}')

train(loader, model, optimizer, criterion, epochs=100)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'torch_geometric.data.data.Data'>

In [134]:
for data in dataset:
    print(f'x shape: {data.x.shape}')
    print(f'edge_index shape: {data.edge_index.shape}')
    print(f'y shape: {data.y.shape}')

x shape: torch.Size([51, 3])
edge_index shape: torch.Size([2, 857605])
y shape: torch.Size([1])
x shape: torch.Size([51, 3])
edge_index shape: torch.Size([0, 857605])
y shape: torch.Size([1])
x shape: torch.Size([51, 3])
edge_index shape: torch.Size([0, 857605])
y shape: torch.Size([1])
x shape: torch.Size([51, 3])
edge_index shape: torch.Size([0, 857605])
y shape: torch.Size([1])
x shape: torch.Size([51, 3])
edge_index shape: torch.Size([0, 857605])
y shape: torch.Size([1])
x shape: torch.Size([51, 3])
edge_index shape: torch.Size([0, 857605])
y shape: torch.Size([1])
x shape: torch.Size([51, 3])
edge_index shape: torch.Size([0, 857605])
y shape: torch.Size([1])
x shape: torch.Size([51, 3])
edge_index shape: torch.Size([0, 857605])
y shape: torch.Size([1])
x shape: torch.Size([51, 3])
edge_index shape: torch.Size([0, 857605])
y shape: torch.Size([1])
x shape: torch.Size([51, 3])
edge_index shape: torch.Size([0, 857605])
y shape: torch.Size([1])
x shape: torch.Size([51, 3])
edge_index 

Edge index excluded from dataloader (used it as a global variable)

In [76]:
edge_index.shape

torch.Size([2, 107])

In [103]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, x, y):
        """
        Args:
            x (Tensor): Tensor of shape (8015, 51, 3) containing the input data.
            labels (Tensor): Tensor of shape (8015,) containing the labels.
        """
        self.x = x.float()  # Ensure x is float32
        self.y = y.float()  # Ensure y is float32
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [104]:
dataset = MyDataset(data_x, data_y)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [105]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)  # Output 1 feature per node

    def forward(self, data):
        global edge_index
        x = data[0]
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(Net, self).__init__()
        self.gcn = GCN(in_channels, hidden_channels)
        self.fc1 = torch.nn.Linear(51, 128)  # Input size should match number of nodes
        self.fc2 = torch.nn.Linear(128, 1)   # Output a single value

    def forward(self, data):
        x = self.gcn(data)
        # print(f"GCN Output Shape: {x.shape}")
        x = x.view(1, -1)
        # print(f"Flattened Output Shape: {x.shape}")
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Model parameters
in_channels = 3
hidden_channels = 16

# Initialize model
model = Net(in_channels, hidden_channels)
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [106]:
def train(loader, model, optimizer, criterion, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in loader:
            optimizer.zero_grad()
            x, y = batch
            x = x.float()  # Ensure x is float32
            y = y.float()  # Ensure y is float32
            # print(f"Batch x Shape: {x.shape}")
            # print(f"Batch y Shape: {y.shape}")
            out = model((x, None))
            # print(out)
            # print(y)
            # print(f"Model Output Shape: {out.shape}")
            y = y.view(out.shape)  # Ensure y matches output shape
            loss = criterion(out, y)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize the model
            total_loss += loss.item()  # Accumulate the loss

        # if epoch % 10 == 0:
        avg_loss = total_loss / len(loader)  # Compute average loss for the epoch
        print(f'Epoch {epoch}, Loss: {avg_loss:.4f}')



train(dataloader, model, optimizer, criterion, epochs=100)

Epoch 0, Loss: 2038343.6008
Epoch 1, Loss: 2016892.1208
Epoch 2, Loss: 2016329.1439
Epoch 3, Loss: 2014444.9475
Epoch 4, Loss: 2015782.4923
Epoch 5, Loss: 2016481.1441
Epoch 6, Loss: 2017809.5966
Epoch 7, Loss: 2015563.5918
Epoch 8, Loss: 2014500.9254


KeyboardInterrupt: 

In [ ]:
# Training loop
def train(data, model, optimizer, criterion, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out.squeeze(), data.y.float())
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Training the model
train(data, model, optimizer, criterion)
